In [6]:
import pandas as pd
df = pd.read_csv("/home/mahdi/Knowledge-Graph/nr-stations-all.csv", nrows=5, encoding="utf-8-sig")
print(df)  # reveals BOM as \\ufeff


   crs  nationalLocationCode       name sixteenCharacterName  \
0  ABE                381300       Aber                 ABER   
1  ACY                380100  Abercynon            ABERCYNON   
2  ABA                398200   Aberdare             ABERDARE   
3  AVY                443500  Aberdovey            ABERDOVEY   
4  ABH                444000   Abererch             ABERERCH   

                                             address      long        lat  \
0  Aber station, Nantgarw Road, Aber, Caerphilly,... -3.229839  51.574961   
1  Abercynon station, Station Road, Abercynon, Rh... -3.327001  51.644706   
2  Aberdare station, Abernant Road, Aberdare, Mid... -3.443099  51.715057   
3  Aberdovey station, Station Road, Aberdovey, Gw... -4.057081  52.543972   
4  Abererch station, Abererch Sands Road, Aberech... -4.374196  52.898600   

                                                 uri  
0  https://www.nationalrail.co.uk/stations/ABE/de...  
1  https://www.nationalrail.co.uk/stations

In [7]:
import pandas as pd

def normalize(df: pd.DataFrame) -> pd.DataFrame:
    # Normalize headers: strip, lowercase, remove BOM
    df.columns = (df.columns
        .str.replace("\ufeff", "", regex=False)
        .str.strip()
        .str.lower()
    )
    # Map likely ID columns to 'id'
    for c in ["id","station_id","code","crs","tiploc"]:
        if c in df.columns:
            if c != "id":
                df = df.rename(columns={c: "id"})
            break
    else:
        raise ValueError(f"No ID column found in headers: {list(df.columns)}")

    # Clean ID values
    df["id"] = df["id"].astype(str).str.replace("\ufeff", "", regex=False).str.strip()
    df.loc[df["id"].isin(["", "nan", "None"]), "id"] = pd.NA

    # # Ensure optional columns exist
    # for c in ["name","lat","lon"]:
    #     if c not in df.columns:
    #         df[c] = pd.NA
    return df

# Example read (utf-8-sig auto-strips BOM)
chunk_station = pd.read_csv("/home/mahdi/Knowledge-Graph/nr-stations-all.csv",
                    chunksize=10_000, encoding="utf-8-sig")

for df_station in chunk_station:
    df_station = normalize(df_station)
    rows_station = df_station.dropna(subset=["id"]).to_dict("records")
    # ... send rows using the guarded Cypher above

 

In [8]:
rows_station[:10]

[{'id': 'ABE',
  'nationallocationcode': '381300',
  'name': 'Aber',
  'sixteencharactername': 'ABER',
  'address': 'Aber station, Nantgarw Road, Aber, Caerphilly, CF83 1AQ',
  'long': -3.229838935,
  'lat': 51.57496069,
  'uri': 'https://www.nationalrail.co.uk/stations/ABE/details.html'},
 {'id': 'ACY',
  'nationallocationcode': '380100',
  'name': 'Abercynon',
  'sixteencharactername': 'ABERCYNON',
  'address': 'Abercynon station, Station Road, Abercynon, Rhondda Cynon Taf, CF45 4SE',
  'long': -3.327000754,
  'lat': 51.644706,
  'uri': 'https://www.nationalrail.co.uk/stations/ACY/details.html'},
 {'id': 'ABA',
  'nationallocationcode': '398200',
  'name': 'Aberdare',
  'sixteencharactername': 'ABERDARE',
  'address': 'Aberdare station, Abernant Road, Aberdare, Mid Glamorgan, CF44 0PU',
  'long': -3.443099147,
  'lat': 51.71505747,
  'uri': 'https://www.nationalrail.co.uk/stations/ABA/details.html'},
 {'id': 'AVY',
  'nationallocationcode': '443500',
  'name': 'Aberdovey',
  'sixteen

In [11]:
# pip install neo4j
from neo4j import GraphDatabase
import csv
# --- 1) Connect ---
URI  = "bolt://localhost:7687"          # or neo4j+s://<your-aura-endpoint>
user = "neo4j"
password= "Mbg!234567"
AUTH = (user,password)            # <- change
host = "bolt://127.0.0.1:7687"


In [12]:

driver = GraphDatabase.driver(URI, auth=AUTH)

# 'stations' is your Python list of dicts (the one you pasted)
stations = rows_station  # <— paste your list here
# --- 2) Constraints (safe to re-run) ---
with driver.session(database="neo4j") as s:
    s.run("""
    CREATE CONSTRAINT station_crs_unique IF NOT EXISTS
    FOR (s:Station) REQUIRE s.crs IS UNIQUE
    """)

# --- 3) Writer (guards null/blank IDs, casts types, maps `long`->`lon`) ---
def write_stations(tx, rows):
    tx.run("""
    UNWIND $rows AS row
    WITH row
    WHERE row.crs IS NOT NULL AND trim(toString(row.crs)) <> ""
    MERGE (s:Station {crs: toString(row.crs)})
    SET  s.nationalLocationCode  = toString(row.nationalLocationCode),
         s.name                  = row.name,
         s.sixteenCharacterName  = row.sixteenCharacterName,
         s.address               = row.address,
         s.uri                   = row.uri,
         // keep your original 'long' if you want, but also store 'lon'
         s.long                  = CASE WHEN row.long IS NULL THEN NULL ELSE toFloat(row.long) END,
         s.lon                   = CASE WHEN row.long IS NULL THEN NULL ELSE toFloat(row.long) END,
         s.lat                   = CASE WHEN row.lat  IS NULL THEN NULL ELSE toFloat(row.lat)  END
    """, rows=rows)

# --- 4) Batch the write (adjust batch size if needed) ---
BATCH = 1000
for i in range(0, len(stations), BATCH):
    batch = stations[i:i+BATCH]
    with driver.session(database="neo4j") as s:
        s.execute_write(write_stations, batch)

# --- 5) Quick sanity check ---
with driver.session(database="neo4j") as s:
    count = s.run("MATCH (s:Station) RETURN count(s) AS n").single()["n"]
print(f"Loaded stations: {count:,}")


Loaded stations: 0


In [5]:



LINKS_CSV = "/home/mahdi/Knowledge-Graph/nr-station-links.csv"  # 



driver = GraphDatabase.driver(URI, auth=AUTH)

# Ensure node key is unique (safe to re-run)
with driver.session() as s:
    s.run("""
    CREATE CONSTRAINT station_crs IF NOT EXISTS
    FOR (s:Station) REQUIRE s.crs IS UNIQUE
    """)

# --- helpers ---
def fnum(x):
    if x is None: return None
    s = str(x).strip()
    if s == "" or s.lower() in ("nan", "none"): return None
    try: return float(s)
    except ValueError: return None

def normalize_row(r):
    # YOUR HEADERS: 'from', 'to', 'distance'
    return {
        "a": (r.get("from") or "").strip(),
        "b": (r.get("to") or "").strip(),
        "distance_km": fnum(r.get("distance")),   # will be stored as distance_km
        "duration_min": None,
        "line": None,
    }

# --- writers ---
def write_links_bidirectional(tx, rows):
    tx.run("""
    UNWIND $rows AS row
    WITH trim(row.a) AS a, trim(row.b) AS b, row
    WHERE a <> '' AND b <> ''
    MATCH (u:Station {crs:a}), (v:Station {crs:b})
    UNWIND [[u,v],[v,u]] AS pair              // remove this UNWIND if you want one-way links only
    WITH pair[0] AS x, pair[1] AS y, row
    MERGE (x)-[e:CONNECTS_TO]->(y)
    SET e.distance_km = COALESCE(row.distance_km, e.distance_km),
        e.duration_min = COALESCE(row.duration_min, e.duration_min),
        e.line         = COALESCE(row.line, e.line)
    """, rows=rows)

# --- load in batches with debugging ---
BATCH = 5000
buf = []
read_rows = kept_rows = sent_rows = 0
skipped_blank = 0

with open(LINKS_CSV, newline="", encoding="utf-8-sig") as f:
    rdr = csv.DictReader(f)
    headers = rdr.fieldnames
    print("Detected headers:", headers)

    for raw in rdr:
        read_rows += 1
        row = normalize_row(raw)
        if not row["a"] or not row["b"]:
            skipped_blank += 1
            continue
        kept_rows += 1
        buf.append(row)
        if len(buf) == BATCH:
            with driver.session() as s:
                s.execute_write(write_links_bidirectional, buf)
            sent_rows += len(buf)
            buf.clear()

if buf:
    with driver.session() as s:
        s.execute_write(write_links_bidirectional, buf)
    sent_rows += len(buf)

print(f"Rows read: {read_rows:,}")
print(f"Rows kept (non-empty endpoints): {kept_rows:,}")
print(f"Rows skipped (blank endpoints): {skipped_blank:,}")
print(f"Rows sent to DB: {sent_rows:,}")

Detected headers: ['from', 'to', 'distance']
Rows read: 5,802
Rows kept (non-empty endpoints): 5,802
Rows skipped (blank endpoints): 0
Rows sent to DB: 5,802


In [6]:
import pandas as pd
df = pd.read_csv("/home/mahdi/Knowledge-Graph/nr-stations-all.csv", nrows=5, encoding="utf-8-sig")
print("Columns:", [c.encode('unicode_escape').decode() for c in df.columns])  # reveals BOM as \\ufeff


Columns: ['crs', 'nationalLocationCode', 'name', 'sixteenCharacterName', 'address', 'long', 'lat', 'uri']
